In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
pd.options.display.max_columns = None

In [2]:
PATH_DATA_RAW= '../Dataset/Raw/'
PATH_DATA_CLEAN = '../Dataset/Clean/'

# Data Definition

* **hotel**: Type of hotel (Categorical)
* **is_canceled**: Whether the booking was canceled or not (Binary)
* **lead_time**: Number of days between booking date and arrival date (Numeric)
* **arrival_date_year**: The year of the arrival date (Numeric)
* **arrival_date_month**: The month of the arrival date (Categorical)
* **arrival_date_week_number**: The week number of the arrival date (Numeric)
* **arrival_date_day_of_month**: The day of the month of the arrival date (Numeric)
* **stays_in_weekend_nights**: Number of weekend nights stayed or booked to stay at the hotel (Numeric)
* **stays_in_week_nights**: Number of week nights stayed or booked to stay at the hotel (Numeric)
* **adults, children, babies**: Number of guests categorized by age groups
* **adults** = Number of adults
* **children** = Number of children
* **babies** = Number infants

Booking Details:
* **meal**: Type(s) food option(s) included in booking package (Categorical)
* **country & market_segment & distribution_channel** columns provide demographic and customer classification information.
* **is_repeated_guest** column specifies whether a guest is a repeated visitor or not.
* **previous_cancellations** column indicates how many previous bookings were canceled by a guest.
* **previous_bookings_not_canceled** shows how many previous bookings were not canceled by a guest.

Accommodation Details:
* **reserved_room_type** column indicates which type room was originally reserved for each booking.
* **assigned_room_type** mentions which type room was finally assigned for each booking.
* **booking_changes**: Number of changes made to the booking before arrival.
* **deposit_type**: Type of deposit made for the booking (Categorical).
* **agent & company** columns provide relevant information about the travel agency and/or company involved in making the reservation.
* **required_car_parking_spaces**: The number of car parking spaces required by guests for a specific hotel booking.
* **total_of_special_requests**: Total special requests submitted by guests for hotel bookings, covering a wide range of additional needs and preferences.

Additional Information:
* **days_in_waiting_list**: Number of days the booking was on a waiting list before it was confirmed or canceled.
* **customer_type** provides information on types of customers (Categorical)
* **adr:** Average daily rate per room, calculated by dividing the sum of all lodging transactions by the total number of staying nights (Numeric)

In [3]:
df = pd.read_csv(PATH_DATA_RAW+'train.csv')
df.sample(3)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,bookingID
7228,Orlando International Drive/Convention Center ...,1,16,2018,April,15,8,2,3,2,0.0,0,BB,CHE,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,NaN,0,Transient,131.20,0,0,Canceled,2018-03-23,7229
34951,"W New York – Union Square New York, NY",1,194,2018,August,35,22,1,2,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,16.0,NaN,0,Transient,102.25,0,1,Canceled,2018-06-01,34952
9919,Irvine John Wayne Airport/Orange County Courty...,0,6,2018,December,52,18,2,1,2,1.0,0,BB,FRA,Online TA,TA/TO,0,0,0,A,D,0,No Deposit,9.0,NaN,0,Transient,116.00,0,2,Check-Out,2018-12-21,9920


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83293 entries, 0 to 83292
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           83293 non-null  object 
 1   is_canceled                     83293 non-null  int64  
 2   lead_time                       83293 non-null  int64  
 3   arrival_date_year               83293 non-null  int64  
 4   arrival_date_month              83293 non-null  object 
 5   arrival_date_week_number        83293 non-null  int64  
 6   arrival_date_day_of_month       83293 non-null  int64  
 7   stays_in_weekend_nights         83293 non-null  int64  
 8   stays_in_week_nights            83293 non-null  int64  
 9   adults                          83293 non-null  int64  
 10  children                        83290 non-null  float64
 11  babies                          83293 non-null  int64  
 12  meal                            

Dari tipe data:\
Kolom children harusnya adalah int64 bukan float64 karena jumlah anak itu dihitung bukan diukur\
Kolom company dan agent adalah kolom ID, idealnya tipe datanya antara objek atau int\
Kolom reservation_status_date harusnya adalah datetime

Secara umum:\
Data tersebut memiliki 83293 baris dan 33 kolom\
Data tersebut memiliki missing value pada beberapa kolom\
Target untuk diprediksi adalah is_canceled\
Kolom selain is_canceled adalah fitur yang digunakan untuk memprediksi is_canceled

## Checking missing and duplicate value

In [5]:
data_missing_value = df[df.columns[df.isna().any()]].isnull().sum().reset_index()
data_missing_value.columns = ['fitur','missing_count']
data_missing_value['percentage'] = round((data_missing_value['missing_count']/len(df))*100,3)
data_missing_value = data_missing_value.sort_values('percentage', ascending=False).reset_index(drop=True)
data_missing_value

,fitur,missing_count,percentage
0,company,78559,94.316
1,agent,11404,13.691
2,country,346,0.415
3,children,3,0.004


Ada 4 kolom yang memiliki missing value, yaitu:
1. Company
2. Agent
3. Country
4. Children

- Kolom company karena memiliki missing value yang terlalu banyak, maka kolomnya akan di drop
- Kolom agent akan diisi dengan 0 dengan asumsi bahwa orang yang reservasi tidak melewati agent
- Kolom country dan children karena memiliki persentase missing value yang sangat sedikit maka baris2 yang mengandung missing value tersebut akan dihapus

In [6]:
# Drop kolom company
df.drop(columns=['company'], inplace=True)

# Menghapus baris-baris yang berisi missing value pada kolom children dan country
df.dropna(subset=['children'], inplace=True)
df.dropna(subset=['country'], inplace=True)

# Mengisi angka 0 terhadap nilai missing value pada agent
df['agent'] = df['agent'].apply(lambda x: 0 if pd.isnull(x) else x)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82944 entries, 0 to 83292
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           82944 non-null  object 
 1   is_canceled                     82944 non-null  int64  
 2   lead_time                       82944 non-null  int64  
 3   arrival_date_year               82944 non-null  int64  
 4   arrival_date_month              82944 non-null  object 
 5   arrival_date_week_number        82944 non-null  int64  
 6   arrival_date_day_of_month       82944 non-null  int64  
 7   stays_in_weekend_nights         82944 non-null  int64  
 8   stays_in_week_nights            82944 non-null  int64  
 9   adults                          82944 non-null  int64  
 10  children                        82944 non-null  float64
 11  babies                          82944 non-null  int64  
 12  meal                            

In [8]:
df.duplicated().sum()

0

## Change type Data

In [9]:
# Changing children to integer type: 
df['children'] = df['children'].astype(int)

# Changing reservation status date to datetime type: 
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

# Changing agent to integer type: 
df['agent'] = df['agent'].astype(int)

# Change Value data

In [10]:
# Cek range data dari numerical data
df.describe()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,bookingID
count,82944.000000,82944.000000,82944.000000,82944.000000,82944.000000,82944.000000,82944.000000,82944.000000,82944.000000,82944.000000,82944.000000,82944.000000,82944.000000,82944.000000,82944.000000,82944.000000,82944.000000,82944.000000,82944.000000,82944.000000
mean,0.370443,104.214205,2018.157986,27.164557,15.787447,0.931604,2.505437,1.856337,0.103926,0.008042,0.030346,0.087047,0.136839,0.223151,75.024185,2.362136,101.888512,0.061138,0.572085,41642.708056
std,0.482926,106.917129,0.706614,13.608945,8.775334,1.000723,1.913088,0.605626,0.399446,0.100733,0.171538,0.864606,1.543133,0.656692,107.247974,17.857450,48.018623,0.243824,0.793444,24043.330863
min,0.000000,0.000000,2017.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,18.000000,2018.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000,70.000000,0.000000,0.000000,20820.750000
50%,0.000000,69.000000,2018.000000,27.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,95.000000,0.000000,0.000000,41635.500000
75%,1.000000,160.000000,2019.000000,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,154.000000,0.000000,126.000000,0.000000,1.000000,62465.250000
max,1.000000,737.000000,2019.000000,53.000000,31.000000,16.000000,40.000000,55.000000,10.000000,10.000000,1.000000,26.000000,70.000000,21.000000,535.000000,391.000000,508.000000,8.000000,5.000000,83293.000000


Nilai 0 pada adults ini adalah hal yang tidak masuk akal, karena memesan hotel minimal harus dipesan untuk satu orang dewasa\
Baris dengan adulst sama dengan nol akan didrop

In [11]:
df.drop(df[df['adults'] < 1].index, inplace = True)

Nilai 0 pada adr tidak masuk akal, karena adr adalah biaya yang harus dibayarkan. Setiap kamar hotel pasti memiliki biaya sewa per malamnya, sehingga seharusnya nilai 'adr' selalu lebih dari 0. Sehingga baris dengan kolom adr adalah 0 akan dihapus


In [15]:
df.drop(df[df['adr'] == 0].index, inplace = True)

In [16]:
df.describe()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,bookingID
count,81389.000000,81389.000000,81389.000000,81389.000000,81389.000000,81389.000000,81389.000000,81389.000000,81389.000000,81389.000000,81389.000000,81389.000000,81389.000000,81389.000000,81389.000000,81389.000000,81389.000000,81389.000000,81389.000000,81389.000000
mean,0.375223,105.243006,2018.161557,27.132585,15.786826,0.939832,2.525378,1.863876,0.100972,0.007962,0.026330,0.087100,0.124734,0.217106,75.531030,2.381845,103.653891,0.061360,0.570581,41646.139319
std,0.484183,106.963567,0.705913,13.578381,8.776389,0.996926,1.896194,0.475281,0.392028,0.100549,0.160117,0.870134,1.491666,0.631314,107.413906,17.954547,46.553918,0.244303,0.791504,24044.483997
min,0.000000,0.000000,2017.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.260000,0.000000,0.000000,1.000000
25%,0.000000,19.000000,2018.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000,71.100000,0.000000,0.000000,20820.000000
50%,0.000000,71.000000,2018.000000,27.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,95.000000,0.000000,0.000000,41642.000000
75%,1.000000,162.000000,2019.000000,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,155.000000,0.000000,126.000000,0.000000,1.000000,62478.000000
max,1.000000,629.000000,2019.000000,53.000000,31.000000,16.000000,40.000000,4.000000,10.000000,10.000000,1.000000,26.000000,70.000000,16.000000,535.000000,391.000000,508.000000,8.000000,5.000000,83293.000000


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81389 entries, 0 to 83292
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   hotel                           81389 non-null  object        
 1   is_canceled                     81389 non-null  int64         
 2   lead_time                       81389 non-null  int64         
 3   arrival_date_year               81389 non-null  int64         
 4   arrival_date_month              81389 non-null  object        
 5   arrival_date_week_number        81389 non-null  int64         
 6   arrival_date_day_of_month       81389 non-null  int64         
 7   stays_in_weekend_nights         81389 non-null  int64         
 8   stays_in_week_nights            81389 non-null  int64         
 9   adults                          81389 non-null  int64         
 10  children                        81389 non-null  int32         
 11  ba

# Export Data

In [18]:
joblib.dump(df, PATH_DATA_CLEAN+'clean_data.pkl')

['../Dataset/Clean/clean_data.pkl']